In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json

**Read data**

In [2]:
root_path = '/kaggle/input/CORD-19-research-challenge/'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})

In [3]:
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

60596

In [4]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'

In [5]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [6]:
dict_ = {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    
    try:
        content = FileReader(entry)
    except Exception as e:
        continue  # invalid paper format, skip
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['abstract'].append(content.abstract)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)
    
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(get_breaks('. '.join(authors), 40))
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
    # add doi
    dict_['doi'].append(meta_data['doi'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])

Processing index: 0 of 60596
Processing index: 6059 of 60596
Processing index: 12118 of 60596
Processing index: 18177 of 60596
Processing index: 24236 of 60596
Processing index: 30295 of 60596
Processing index: 36354 of 60596
Processing index: 42413 of 60596
Processing index: 48472 of 60596
Processing index: 54531 of 60596
Processing index: 60590 of 60596


In [7]:
# Combine abstract and body text
df_covid['abstract_body_text']   = df_covid['abstract']+' '+df_covid['body_text']

In [8]:
# Process keywords from external databases
path = 'externaldataset/*.txt'
keyword_files = glob.glob(path)
all_keyword =[]
for f in keyword_files:
    file_object  = open(f, 'r')
    keyword = file_object.read()
    keyword_arr = keyword.split('\n')
    keyword_arr_clean = [i.lower().replace('/', ' ').replace('^+^', '').replace(':','').replace('-','') for i in keyword_arr]
    all_keyword.append(keyword_arr_clean)

In [9]:
keywords = [item.strip().lower() for sublist in all_keyword for item in sublist if item]
keywords_uni = set(keywords)
len(keywords_uni)

15814

In [10]:
# Match keywords
import re
def match_keyword(text):
    return [dis for dis in keywords_uni if dis in text.lower()] 

In [ ]:
# !pip install swifter

In [1]:
# Save keyword matches to files of 500 documents each. These files will be used in 3.lda_visualization.ipynb.
# This process takes a long time. Change the value of count. You can start from where you had left.
import time
import swifter
n = 500  #chunk row size
list_df = [df_covid[i:i+n] for i in range(0,df_covid.shape[0],n)]
count=60 # Choose your count, you can start from where you left.
for i in range(count, len(list_df)):
    print('processing..', count)
    df = list_df[i]
    df['keyword_match']=df.swifter.apply(lambda row: match_keyword(row['abstract_body_text']),axis=1)
    df[['paper_id','keyword_match']].to_csv('keyword_match_data/df_'+str(count)+'.csv', index=False)
    count=count+1